In [ ]:
!pip install selenium
!pip install webdriver_manager

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00


In [ ]:
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!sudo dpkg -i google-chrome-stable_current_amd64.deb
!sudo apt --fix-broken install

--2024-10-22 18:39:29--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 74.125.197.91, 74.125.197.136, 74.125.197.190, ...
Connecting to dl.google.com (dl.google.com)|74.125.197.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 111386844 (106M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb’

google-chrome-stabl 100%[===================>] 106.23M   215MB/s    in 0.5s    

2024-10-22 18:39:30 (215 MB/s) - ‘google-chrome-stable_current_amd64.deb’ saved [111386844/111386844]

Selecting previously unselected package google-chrome-stable.
(Reading database ... 123622 files and directories currently installed.)
Preparing to unpack google-chrome-stable_current_amd64.deb ...
Unpacking google-chrome-stable (130.0.6723.58-1) ...
dpkg: dependency problems prevent configuration of google-chrome-stable:
 google-chrome-stable depends on libvulkan1; however:
  Package li

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time


# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.binary_location = "/usr/bin/google-chrome"

# Set up WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# URL for the Airbnb listing reviews
url = 'https://www.airbnb.co.in/rooms/54319987/reviews'
driver.get(url)

# Give the page more time to load initially
time.sleep(10)

# review_section = driver.find_element(By.CSS_SELECTOR, "[data-testid='pdp-reviews-modal-scrollable-panel']")
review_sec  = driver.find_element(By.CLASS_NAME, '_17itzz4')
last_height = driver.execute_script("return arguments[0].scrollHeight", review_sec)

while True:
    # Scroll to the bottom of the scrollable div
    driver.execute_script("arguments[0].scrollBy(0, "+str(last_height)+");", review_sec)
    # Wait for new content to load (adjust the sleep time if necessary)
    time.sleep(5)
    # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    new_height = driver.execute_script("return arguments[0].scrollHeight", review_sec)

     # Exit the loop if no new content is loaded (reached the bottom)
    if new_height == last_height:
        print("No more new content. Stopping scroll.")
        break
    last_height = new_height

# Capture the page source and close the browser
page_source = driver.page_source
driver.quit()

# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

# Find review elements by exploring their actual HTML structure
reviews = soup.find_all('div', 'r1are2x1 atm_gq_1vi7ecw dir dir-ltr')

# Debugging check
if not reviews:
    print("No reviews found. Adjust the selectors.")
else:
    print(f"Total reviews found: {len(reviews)}")

# Constants for hotel info
hotel_name = "Tigers Landing Guest House"
overall_rating = "4.75/5"  # Update to reflect "4.75/5" format

# Prepare the data structure for storing all review details
review_data = []

for idx, review in enumerate(reviews, start=1):
    try:
        # Extract review text
        text = review.find('span', 'l1h825yc atm_kd_19r6f69_24z95b atm_kd_19r6f69_1xbvphn_1oszvuo dir dir-ltr').get_text(strip=True)

        # Extract rating and format as 'Rating, <star> star(s)'
        rating_element = review.find('span', 'a8jt5op atm_3f_idpfg4 atm_7h_hxbz6r atm_7i_ysn8ba atm_e2_t94yts atm_ks_zryt35 atm_l8_idpfg4 atm_mk_stnw88 atm_vv_1q9ccgz atm_vy_t94yts dir dir-ltr')
        rating = rating_element.get_text(strip=True).split(" star")[0][-1] + "/5" if rating_element else "N/A"

        # Append data in the specified format without the Hotel Details column
        review_data.append((idx, text, rating, hotel_name, overall_rating))
    except AttributeError as e:
        print(f"Could not retrieve data for review {idx}: {e}")

# Prepare data for DataFrame without the Hotel Details column
data = {
    "Review Number": [idx for idx, text, rating, hotel_name, overall_rating in review_data],
    "Review (Text)": [text for idx, text, rating, hotel_name, overall_rating in review_data],
    "Individual Rating": [rating for idx, text, rating, hotel_name, overall_rating in review_data],
    "Hotel Name": [hotel_name for idx, text, rating, hotel_name, overall_rating in review_data],
    "Overall Rating": [overall_rating for idx, text, rating, hotel_name, overall_rating in review_data]
}

# Save to Excel
df = pd.DataFrame(data)
output_path = 'Airbnb_Reviews.xlsx'
df.to_excel(output_path, index=False)

print(f"Data saved to {output_path}")

No more new content. Stopping scroll.
Total reviews found: 110
Data saved to Airbnb_Reviews.xlsx
